In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import time
import os

In [2]:
data_dir = '/app/data/store'

classes = ['W','S','A','D','-']

def vis_layers(folder, inputs):
    
    if not os.path.exists('./' + folder):
        os.makedirs('./' + folder)
        
    outputs = []
    names = []
    for layer in modulelist[1:]:
        inputs = layer(inputs)
        outputs.append(inputs)
        names.append(str(layer).partition('(')[0])

    for i, name, image in zip(range(len(names)), names, outputs):
        final = torch.cat((image[0].data), 0)
        print(name, '  ', image[0].size(), '   ', final.size())
        plt.imsave(fname='./{0}/{1}_{2}.png'.format(folder, i, name), arr=final, cmap='gray')
        
        
def accuracy(inputs, labels):
    success = 0
    for i in range(10):
        rez = []
        for index, key in enumerate(inputs):
            rez.append(key[i].max(0)[1].data[0])
        l = labels[i].data.view(1,10)
        f = torch.cuda.LongTensor(rez).view(1,10)
        
        if(torch.equal(l,f)): success+=1
    
    percent = success / len(inputs)
    return percent


def test_accuracy(train=True):
    model.eval()
    total_acc = 0
    for i, data in enumerate(test_dataset):
        inputs = Variable(data['image'].cuda())
        labels = Variable(data['labels'].cuda())
        layers = model(inputs)
        final = layers['final']

        total_acc += accuracy(final,labels)
    model.train()
    return total_acc / len(test_dataset)


def name_classes(value):
    a= ''
    for i in range(len(value)):
        a += classes[int(value[i])]
    return a

In [3]:
batch_size = 10

In [4]:
class KapchaDataset(Dataset):
    """Kapcha dataset."""

    def __init__(self, root_dir, img_dir, csv_dir, length, first_idx=0):
        """
        Args:
            root_dir (string): Root directory.
            img_dir  (string): Directory with all images.
            csv_dir  (string): Directory with labels.
            length      (int): Total length of dataset.
            first_idx   (int): First data index.
        """
        self.root_dir   = root_dir
        self.img_dir    = os.path.join(root_dir, img_dir)
        self.csv_dir    = os.path.join(root_dir, csv_dir)
        self.first_idx  = first_idx
        self.length     = length
        
        self.mean       = 0.079210128784179684
        self.std        = 0.083445704142252608

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        idx = self.first_idx + idx

        img_name    = os.path.join(self.img_dir, '{0}.jpg'.format(idx))
        image       = Image.open(img_name)
        image       = np.array(image).astype('float32') / 255
#         image       -= self.mean
#         image       /= self.std
        
        image       = image[:, :, 0].reshape(1, 41, 265)
        image       = torch.from_numpy(image)

        label_name  = os.path.join(self.csv_dir, '{0}.csv'.format(idx))
        labels      = np.genfromtxt(label_name, delimiter=',').astype('int64')
        labels      = torch.from_numpy(labels)

        sample = {'image': image, 'labels': labels, 'idx': idx}

        return sample

train_dataset   = KapchaDataset(root_dir=data_dir,
                                img_dir='resized1',
                                csv_dir='order',
                                length=12000,
                                first_idx=0)

train_dataset   = DataLoader(train_dataset,
                             batch_size=batch_size,
                             shuffle=True,
                             num_workers=0)

test_dataset    = KapchaDataset(root_dir=data_dir,
                                img_dir='resized1',
                                csv_dir='order',
                                length=2360,
                                first_idx=12000)

test_dataset    = DataLoader(test_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=0)

In [5]:
class Parser(nn.Module):

    def __init__(self):
        super(Parser, self).__init__()
        
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, 3),
            torch.nn.BatchNorm2d(16),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(2, stride=2),
            
            torch.nn.Conv2d(16, 32, 3),
            torch.nn.BatchNorm2d(32),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(2, stride=2),
            
            torch.nn.Conv2d(32, 64, 3),
            torch.nn.BatchNorm2d(64),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(2, stride=2),
        )
             
        self.middle = torch.nn.Sequential(
            torch.nn.Linear(64 * 3 * 31, 1000),
            torch.nn.BatchNorm1d(1000),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(p=0.5),
            
            torch.nn.Linear(1000, 100),
            torch.nn.BatchNorm1d(100),
            torch.nn.LeakyReLU(),
        )
        
        self.classification = torch.nn.Sequential(
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5),
            torch.nn.Linear(100, 5)
        )
        
        
    def forward(self, inputs):
        
        features = self.features(inputs)
        re_pool3 = features.view(-1 , 64 * 3 * 31)
        
        linear2 = self.middle(re_pool3)
        
        final = []
        for index, layer in enumerate(self.classification):
            final.append(layer(linear2))
        
        return {'final': final}


In [10]:
model = Parser().cuda()
model

Parser(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (middle): Sequential(
    (0): Linear(in_features=5952, out_features=1000, bias=True)
    (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affin

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [14]:
model.train()

for epoch in range(1):
    run_loss = 0
    run_acc = 0
    total_acc= 0
    for i, data in enumerate(train_dataset):
        inputs = Variable(data['image'].cuda())
        labels = Variable(data['labels'].cuda())
        
        layers = model(inputs)
        final = layers['final']
        
        losses = []
        for char_num in range(10):
            loss = criterion(final[char_num], labels[:,char_num])
            losses.append(loss)
        
        total_loss = sum(losses)
        acc = accuracy(final,labels)
        total_acc += acc
        run_loss += total_loss
        run_acc += acc
        
        if(i != 0 and i % 100 == 0):
            print('{0} : {1}, Loss= {2}, Acc:{3}, TestAcc: {4}'.format(i, len(train_dataset), total_loss.data[0], total_acc / 100,test_accuracy()))
            total_acc = 0
            
        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad() 
    
    print('Epoch: {0},  Loss:{1}, Acc:{2}, TestAcc: {3}'.format(epoch, run_loss.data[0] / len(train_dataset), run_acc / 1200,test_accuracy()))

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


100 : 1200, Loss= 5.223594665527344, Acc:0.05000000000000001, TestAcc: 0.04406779661016943
200 : 1200, Loss= 3.02164363861084, Acc:0.179, TestAcc: 0.17415254237288164
300 : 1200, Loss= 2.3282227516174316, Acc:0.3800000000000001, TestAcc: 0.6326271186440676
400 : 1200, Loss= 3.2919750213623047, Acc:0.595, TestAcc: 0.7771186440677975
500 : 1200, Loss= 2.8729798793792725, Acc:0.696, TestAcc: 0.7504237288135599
600 : 1200, Loss= 2.440894603729248, Acc:0.7410000000000001, TestAcc: 0.7343220338983049
700 : 1200, Loss= 1.0605435371398926, Acc:0.7920000000000001, TestAcc: 0.8923728813559342
800 : 1200, Loss= 1.2861864566802979, Acc:0.8150000000000003, TestAcc: 0.8559322033898322
900 : 1200, Loss= 0.5938747525215149, Acc:0.8360000000000004, TestAcc: 0.9377118644067811
1000 : 1200, Loss= 0.3396875560283661, Acc:0.8459999999999998, TestAcc: 0.9038135593220359
1100 : 1200, Loss= 0.27643871307373047, Acc:0.8549999999999999, TestAcc: 0.8525423728813579


/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: 0,  Loss:2.255427360534668, Acc:0.637499999999996, TestAcc: 0.9559322033898316


In [24]:
torch.save(model.state_dict(), './models/{0}.pt'.format('9_gpu'))
model.cpu()
torch.save(model.state_dict(), './models/{0}.pt'.format('9_cpu'))
model.cuda()

Parser(
  (features): Sequential(
    (0): Conv2d (1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU(0.01)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Conv2d (16, 32, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (6): LeakyReLU(0.01)
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (8): Conv2d (32, 64, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.01)
    (11): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (middle): Sequential(
    (0): Linear(in_features=5952, out_features=1000)
    (1): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU(0.01)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=1000, out_features=100)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=Tru

In [45]:
model.load_state_dict(torch.load('./models/{0}.pt'.format('7_cpu')))

In [13]:
it = iter(test_dataset)
data = it.next()
          
inputs = Variable(data['image'].cuda())
labels = Variable(data['labels'].cuda())

layers = model(inputs)

final = layers['final']

losses = []
for char_num in range(10):
    loss = criterion(final[char_num], labels[:,char_num])
    losses.append(loss)

total_loss = sum(losses)
total_loss.backward()
optimizer.step()
optimizer.zero_grad() 

print('Total loss: {0}'.format(total_loss.data[0]))

Total loss: 16.26456069946289


In [26]:
""" Сохранение некорректно распознанных капч"""

model.eval()
for index, data in enumerate(train_dataset):
    idx = data['idx']
    inputs = Variable(data['image'].cuda())
    labels = Variable(data['labels'].cuda())

    layers = model(inputs)

    final = layers['final']

    for number in range(batch_size):
        rez = []
        for i in range(10):
            rez.append(final[i][number].max(0)[1].data[0])

        f = torch.cuda.LongTensor(rez)
        if(not torch.equal(labels[number].data.view(1,10), f.view(1, 10))):
            plt.imsave(fname='./errors/{0}_{1}.png'.format(idx[number],name_classes(f.view(10))), arr=inputs.data[number][0], cmap='gray')

In [15]:
""" Визуализация слоев"""

it = iter(train_dataset)

value = it.next()
value = Variable(value['image'].cuda())

value.data.resize_(batch_size,1, 58, 372)
print('input size',value.size())

modulelist = list(model.features.modules())

vis_layers('visual/layers/5', value)


input size torch.Size([10, 1, 58, 372])
Conv2d     torch.Size([16, 56, 370])     torch.Size([896, 370])
BatchNorm2d    torch.Size([16, 56, 370])     torch.Size([896, 370])
LeakyReLU    torch.Size([16, 56, 370])     torch.Size([896, 370])
MaxPool2d    torch.Size([16, 28, 185])     torch.Size([448, 185])
Conv2d     torch.Size([32, 26, 183])     torch.Size([832, 183])
BatchNorm2d    torch.Size([32, 26, 183])     torch.Size([832, 183])
LeakyReLU    torch.Size([32, 26, 183])     torch.Size([832, 183])
MaxPool2d    torch.Size([32, 13, 91])     torch.Size([416, 91])
Conv2d     torch.Size([64, 11, 89])     torch.Size([704, 89])
BatchNorm2d    torch.Size([64, 11, 89])     torch.Size([704, 89])
LeakyReLU    torch.Size([64, 11, 89])     torch.Size([704, 89])
MaxPool2d    torch.Size([64, 5, 44])     torch.Size([320, 44])


In [8]:
it = iter(train_dataset)
data = it.next()
          
inputs = Variable(data['image'].cuda())
labels = Variable(data['labels'].cuda())

layers = model(inputs)
print(layers.size())

torch.Size([10, 64, 3, 31])
